In [ ]:
import rdkit
from rdkit.Chem import AllChem
import pandas as pd
import os
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit import Chem
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import Draw
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit import DataStructs
from tqdm import tqdm

In [ ]:
import json
with open('../../sup_info/crossdock2020_duplicated_uniprotId_map_smiles_in_trainset.json','r') as f:
    crossdock2020_duplicated_uniprotId_map_smiles_in_trainset = json.load(f)

In [ ]:

def get_trainset_scaffold(x,ref_map):
    uniprot_id  = x['UniprotID']
    if uniprot_id not in ref_map:
        return []
    else:
        all_find_scaffold = [i[1:-1] for i in x.Finded_Scaffolds[1:-1].split(', ')]
        all_find_scaffold = [Chem.MolToSmiles(Chem.MolFromSmiles(i)) for i in all_find_scaffold if i != '']
        ref_scaffolds = [Chem.MolToSmiles(Chem.MolFromSmiles(i)) for i in ref_map[uniprot_id]['scaffold'] if i != '']
        list_dup_scaffolds = list(set(all_find_scaffold).intersection(set(ref_scaffolds)))
        return list_dup_scaffolds
def get_trainset_smiles(x,ref_map):
    uniprot_id  = x['UniprotID']
    if uniprot_id not in ref_map:
        return []
    else:
        all_find_smiles = [i[1:-1] for i in x.Finded_Smiles[1:-1].split(', ')]
        all_find_smiles = [Chem.MolToSmiles(Chem.MolFromSmiles(i)) for i in all_find_smiles if i != '']
        ref_smiles = [Chem.MolToSmiles(Chem.MolFromSmiles(i)) for i in ref_map[uniprot_id]['smiles'] if i != '']
        list_dup_smiles = list(set(all_find_smiles).intersection(set(ref_smiles)))
    
        return list_dup_smiles
# map nM to pIC50
def get_pAffinity(x):
    '''
    transfer nM to pIC50
    '''
    if not isinstance(x, (int, float)):
        raise TypeError("Input Must Be a Number")
    x = x * 1e-9
    
    x = -np.log10(x)
    return x



## get affinity map info 

In [ ]:

from collections import defaultdict
affinity_info_map = defaultdict(dict)
base_dir = '/home/datahouse1/caoduanhua/MolGens/SelfConstructedBenchmark/MolGenBench_Version1'
for uniprot_id in tqdm(os.listdir(base_dir),total= len(os.listdir(base_dir))):
    # serise_ids = os.listdir(os.path.join(base_dir,uniprot_id,'Hit_to_Lead_Results'))
    # for serise_id in serise_ids:
    ref_actives_path = os.path.join(base_dir,uniprot_id,'reference_active_molecules',f'{uniprot_id}_reference_active_molecules.sdf')
    ref_actives = Chem.SDMolSupplier(ref_actives_path)
    serise_id_map ={}
    for mol in ref_actives:
        mol_name = mol.GetProp('_Name')
        mol_serise_id = mol_name.split('_')[1]
        # if mol_serise_id == serise_id:
        serise_id_string = uniprot_id + '_' + mol_serise_id
        affinity_info_map[serise_id_string][Chem.MolToSmiles(mol)] = mol.GetProp('Affinity')
        

In [ ]:
# len(affinity_info_map.keys())
# all_hit2lead_serise_id_name = 
import glob 
all_hit2lead_serise_id_name = [x.split('/')[-1] for x in glob.glob(f'{base_dir}/*/Round1/Hit_to_Lead_Results/*')]
filtered_keys = []
for key in affinity_info_map.keys():
    if key.split('_')[-1] in all_hit2lead_serise_id_name:
        filtered_keys.append(key)
print(len(filtered_keys))

In [ ]:

def temp_func(info_map):
    sorted_dict = {k:get_pAffinity(float(v)) for k, v in sorted(info_map.items(), key=lambda item: float(item[1]),reverse=False) if float(v) >0.0  }
    return sorted_dict
all_delta = [max( temp_func(affinity_info_map[key]).values()) -min( temp_func(affinity_info_map[key]).values()) for key in filtered_keys]

In [ ]:
# affinity_info_map.keys()
# P53779_Sries20890
temp_dict= affinity_info_map['P53779_Sries20890']
# 计算nM到pIC50
def get_pAffinity(x):
    '''
    transfer nM to pIC50
    '''
    if not isinstance(x, (int, float)):
        raise TypeError("Input Must Be a Number")
    x = x * 1e-9
    
    x = -np.log10(x)
    return x
## this version is min max normalize
def normalize_affinty_map(info_map):
    """
    Normalize the values in the dictionary to a range between 0 and 1.
    Using max normalization.
    Args:
        info_map (dict): A dictionary with values to be normalized.
    Returns:    
        dict: A dictionary with normalized values.
    
    """
    # Sort the dictionary by values in ascending order
    sorted_dict = {k:get_pAffinity(float(v)) for k, v in sorted(info_map.items(), key=lambda item: float(item[1]),reverse=False) if float(v) >=0.0  }
    

    # Normalize the values
    # min_value = min(sorted_dict.values())
    max_value = max(sorted_dict.values())
    min_value = min(sorted_dict.values())
    normalized_dict = {k: ((v - min_value)/ (max_value-min_value))  for k, v in sorted_dict.items()}

    return normalized_dict
def get_find_smiles_affinity(x,affinity_info_map):
    """"
    
    Get the affinity values for the found smiles in the input data.
    Args:
        x (pd.Series): A pandas Series containing 'Finded_Smiles' and ' 
        UniprotID'.
        affinity_info_map (dict): A dictionary mapping UniprotID to another
        dictionary of smiles to affinity values.
    Returns:
        list: A list of affinity values corresponding to the found smiles.
        
        
        """
    
    # print(x)
    find_smiles = x.Finded_Smiles
    uniprot_serise_id = x.UniprotID
    affinity_info = affinity_info_map[uniprot_serise_id]
    affinity_info = normalize_affinty_map(affinity_info)
    affinitys=[]
    for smiles in find_smiles:
        if smiles in affinity_info:
            affinitys.append(affinity_info[smiles])
        else:
            affinitys.append('None')
        
    return affinitys

In [ ]:
# get_find_smiles_rank_in_sampling
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
all_result_dir = 'the dir from hit_info_preprocess_h2l.py'
save_dir = 'your save dir'
for round_num in [1,2,3]:
    all_result_path = f'{all_result_dir}/Round{round_num}'
    all_results = os.listdir(all_result_path)
    all_results_pd = []

    for temp_path in all_results:
        model_name = os.path.splitext(temp_path)[0]
        temp_path = os.path.join(all_result_path, temp_path)
        temp_pd = pd.read_csv(temp_path)
        temp_pd['ModelName'] = [model_name]*len(temp_pd)
        all_results_pd.append(temp_pd)
    merged_result_pd = pd.concat(all_results_pd,axis = 0)#['ModelName'].value_counts()

    merged_result_pd['Finded_Smiles'] = merged_result_pd['Finded_Smiles'].apply(lambda x: [i[1:-1] for i in x[1:-1].split(', ')])
    merged_result_pd['Finded_Smiles_Affinity'] = merged_result_pd.apply(lambda x: get_find_smiles_affinity(x,affinity_info_map),axis = 1)
    merged_result_pd['Finded_Smiles_Affinity'] = merged_result_pd['Finded_Smiles_Affinity'].apply(lambda x: [i for i in x if i!='None' and i!=np.nan])
    merged_result_pd['Finded_Smiles_Num'] = merged_result_pd['Finded_Smiles_Affinity'].apply(lambda x : len(x))
    merged_result_pd['Finded_Smiles_Affinity_mean'] = merged_result_pd['Finded_Smiles_Affinity'].apply(lambda x :  np.mean( [float(i) for i in x if i != 'None']))
    merged_result_pd['NumberHits'] =  merged_result_pd['Finded_Smiles_Affinity'].apply(lambda x :  len([ float(i) for i in x if i != 'None']))
    merged_result_pd['Protein_in_Crossdock2020'] = merged_result_pd['UniprotID'].apply(lambda x: x.split('_')[0] in crossdock2020_duplicated_uniprotId_map_smiles_in_trainset)

    # save all proteins
    affinity_pd = merged_result_pd[merged_result_pd['Finded_Smiles_Num'] > 0][['ModelName','Finded_Smiles_Affinity_mean','UniprotID','NumberHits']]
    # affinity_pd= affinity_pd[affinity_pd['Protein_in_Crossdock2020']]
    affinity_pd = affinity_pd.groupby('ModelName')[['UniprotID','Finded_Smiles_Affinity_mean','NumberHits']].agg(list).reset_index()

    affinity_pd['Serise_Num'] = affinity_pd['NumberHits'].apply(lambda x: len(x))
    affinity_pd['Finded_Smiles_Affinity_mean'] = affinity_pd.apply(lambda x: sum([i*j for (i,j) in zip(x.Finded_Smiles_Affinity_mean,x.NumberHits)])/sum(x.NumberHits),axis = 1)
    affinity_pd['NumberHits_sum'] = affinity_pd['NumberHits'].apply(lambda x: np.sum(x))
    affinity_pd['NumberHits_Mean'] = affinity_pd['NumberHits'].apply(lambda x: np.mean(x))
    # affinity_pd
    affinity_pd.sort_values('Finded_Smiles_Affinity_mean')[['ModelName','NumberHits_sum','Finded_Smiles_Affinity_mean','NumberHits_Mean','Serise_Num']].to_csv(f'{save_dir}/round{round_num}_all.csv')

    # save seen proteins
    affinity_pd = merged_result_pd[merged_result_pd['Finded_Smiles_Num'] > 0][['ModelName','Finded_Smiles_Affinity_mean','UniprotID','NumberHits','Protein_in_Crossdock2020']]
    affinity_pd= affinity_pd[affinity_pd['Protein_in_Crossdock2020']]
    affinity_pd = affinity_pd.groupby('ModelName')[['UniprotID','Finded_Smiles_Affinity_mean','NumberHits']].agg(list).reset_index()

    affinity_pd['Serise_Num'] = affinity_pd['NumberHits'].apply(lambda x: len(x))
    affinity_pd['Finded_Smiles_Affinity_mean'] = affinity_pd.apply(lambda x: sum([i*j for (i,j) in zip(x.Finded_Smiles_Affinity_mean,x.NumberHits)])/sum(x.NumberHits),axis = 1)
    affinity_pd['NumberHits_sum'] = affinity_pd['NumberHits'].apply(lambda x: np.sum(x))
    affinity_pd['NumberHits_Mean'] = affinity_pd['NumberHits'].apply(lambda x: np.mean(x))
    # affinity_pd
    affinity_pd.sort_values('Finded_Smiles_Affinity_mean')[['ModelName','NumberHits_sum','Finded_Smiles_Affinity_mean','NumberHits_Mean','Serise_Num']].to_csv(f'{save_dir}/round{round_num}_seen.csv')

    # save unseen proteins
    affinity_pd = merged_result_pd[merged_result_pd['Finded_Smiles_Num'] > 0][['ModelName','Finded_Smiles_Affinity_mean','UniprotID','NumberHits','Protein_in_Crossdock2020']]
    affinity_pd= affinity_pd[~affinity_pd['Protein_in_Crossdock2020']]
    affinity_pd = affinity_pd.groupby('ModelName')[['UniprotID','Finded_Smiles_Affinity_mean','NumberHits']].agg(list).reset_index()

    affinity_pd['Serise_Num'] = affinity_pd['NumberHits'].apply(lambda x: len(x))
    affinity_pd['Finded_Smiles_Affinity_mean'] = affinity_pd.apply(lambda x: sum([i*j for (i,j) in zip(x.Finded_Smiles_Affinity_mean,x.NumberHits)])/sum(x.NumberHits),axis = 1)
    affinity_pd['NumberHits_sum'] = affinity_pd['NumberHits'].apply(lambda x: np.sum(x))
    affinity_pd['NumberHits_Mean'] = affinity_pd['NumberHits'].apply(lambda x: np.mean(x))
    # affinity_pd
    affinity_pd.sort_values('Finded_Smiles_Affinity_mean')[['ModelName','NumberHits_sum','Finded_Smiles_Affinity_mean','NumberHits_Mean','Serise_Num']].to_csv(f'{save_dir}/round{round_num}_unseen.csv')


## show the result

In [ ]:

def get_result_data(base_dir,type_name):

    data_list = []
    for round_num in [1,2,3]:
        round_path = os.path.join(base_dir,f'round{round_num}_{type_name}.csv')
        data = pd.read_csv(round_path)
        data['round'] = [round_num]*len(data)
        
        data_list.append(
            data
            )
        # [['ModelName','NumberHits_sum','Finded_Smiles_Affinity_mean','NumberHits_Mean','Serise_Num']]
    shapemol_data = pd.DataFrame({
        'ModelName':'ShapeMol',
        'NumberHits_sum': [0, 0, 0],
    'Finded_Smiles_Affinity_mean': [0, 0, 0],
        'NumberHits_Mean': [0, 0, 0],
        'Serise_Num': [0, 0, 0],
        'round': [1, 2, 3]
    })
    # shapemol_data
    repeats_data = pd.concat(data_list,axis = 0)
    merged_data =repeats_data
    # merged_data = pd.concat([repeats_data, shapemol_data], axis=0, ignore_index=True)

    data = pd.DataFrame(merged_data.groupby('ModelName').agg(['mean','std']))
    return data

def plot_fig(data,x_feature,y_feature,type_name,legend=True):
        
    plt.figure(figsize=(8, 6))
    data['modelname'] = data.index

    # 创建颜色和标记映射
    color_list = sns.color_palette("Set2", n_colors=7)
    # markers = ['o', 's', '^', 'v', 'p', '*', 'X']  # 不同的标记形状
    markers = ['o', 'o', 'o', 'o', 'o', 'o', 'o']  # 不同的标记形状

    model_names = sorted(data['modelname'].unique())
    color_map = {name: color_list[i % len(color_list)] for i, name in enumerate(model_names)}
    marker_map = {name: markers[i % len(markers)] for i, name in enumerate(model_names)}

    # 为每个模型单独绘制
    for model in model_names:
        model_data = data[data['modelname'] == model]
        plt.scatter(
            x=model_data[(x_feature, 'mean')],
            y=model_data[(y_feature, 'mean')],
            color=color_map[model],
            marker=marker_map[model],
            s=200,
            alpha=0.8,
            edgecolor="w",
            linewidth=0.8,
            label=model
        )

    plt.ylabel(y_feature,fontsize=14)
    plt.xlabel('Number of Series',fontsize=14)
    plt.title(type_name,fontsize = 14)
    plt.grid(True, linestyle='--', alpha=0.7)
    if legend:
        plt.legend(fontsize =14)
        plt.tight_layout()

        plt.savefig(f'{save_dir}/hit2lead_use_ref_rank_{type_name}_protein_{y_feature}_legend.svg',bbox_inches='tight',dpi=660,format='svg')
    else:
        # plt.legend()
        plt.tight_layout()

        plt.savefig(f'{save_dir}/hit2lead_use_ref_rank_{type_name}_protein_{y_feature}.svg',bbox_inches='tight',dpi=660,format='svg')
        


In [ ]:
base_dir = save_dir
# [['ModelName','NumberHits_sum','Finded_Smiles_Affinity_mean','NumberHits_Mean','Serise_Num']]

x_feature_list  = ['NumberHits_sum','Serise_Num']
y_feature_list = ['Finded_Smiles_Affinity_mean','NumberHits_Mean']
for x_feature ,y_feature in zip(x_feature_list ,y_feature_list):
    for type_name in ['all','seen','unseen']:
        data = get_result_data(base_dir,type_name)
        data.to_csv(f'{save_dir}/hit2lead_use_ref_rank_{type_name}_repeats.csv')
        plot_fig(data,x_feature,y_feature,type_name)
        plot_fig(data,x_feature,y_feature,type_name,legend=False)

## Count the number of active smiles at different affinity threshold

In [ ]:
def get_protein_type_data(merged_result_pd,protein_type='all'):
    if protein_type =='all':
        print('all')
        
        merge_all_affinity_pd = merged_result_pd[merged_result_pd['Finded_Smiles_Num']>0][['UniprotID','Finded_Smiles_Affinity','Finded_Smiles_Num','ModelName']].groupby('ModelName').agg(list).reset_index()
    elif protein_type =='seen':
        print('seen')
        merged_result_pd = merged_result_pd[merged_result_pd['Protein_in_Crossdock2020']]
        merge_all_affinity_pd = merged_result_pd[merged_result_pd['Finded_Smiles_Num']>0][['UniprotID','Finded_Smiles_Affinity','Finded_Smiles_Num','ModelName']].groupby('ModelName').agg(list).reset_index()
    elif protein_type =='unseen':
        print('unseen')
        merged_result_pd = merged_result_pd[~merged_result_pd['Protein_in_Crossdock2020']]
        merge_all_affinity_pd = merged_result_pd[merged_result_pd['Finded_Smiles_Num']>0 & ~merged_result_pd['Protein_in_Crossdock2020']][['UniprotID','Finded_Smiles_Affinity','Finded_Smiles_Num','ModelName']].groupby('ModelName').agg(list).reset_index()
    else:
        raise ValueError('error protein type name')
    merge_all_affinity_pd['Finded_Smiles_Affinity_max_in_series'] = merge_all_affinity_pd['Finded_Smiles_Affinity'].apply(lambda x :[max(i) for i in x] )
    merge_all_affinity_pd['Finded_Smiles_Affinity'] = merge_all_affinity_pd['Finded_Smiles_Affinity'].apply(lambda x :sum(x,[]) )
    return merge_all_affinity_pd
def get_count_data(merge_all_affinity_pd,col_name='Finded_Smiles_Affinity'):
    # 展开数据
    exploded_data = []
    for index, row in merge_all_affinity_pd.iterrows():
        model_name = row['ModelName']
        affinity_list = row[col_name]
        for affinity in affinity_list:
            exploded_data.append({'ModelName': model_name, 'Affinity': affinity})

    exploded_df = pd.DataFrame(exploded_data)

    # 定义阈值
    thresholds = [0.5, 0.8, 0.9, 0.95, 0.99]

    # 计算每个模型在不同阈值下的数量
    results = []
    models = exploded_df['ModelName'].unique()

    for model in models:
        model_data = exploded_df[exploded_df['ModelName'] == model]['Affinity']
        total_count = len(model_data)
        
        # 计算大于每个阈值的数量
        counts = {}
        for threshold in thresholds:
            count_above = len(model_data[model_data >= threshold])
            counts[f'>={threshold}'] = count_above
        
        # 添加到结果
        counts['Model'] = model
        counts['Total_Count'] = total_count
        results.append(counts)

    # 转换为DataFrame
    results_df = pd.DataFrame(results)

    # 设置模型为索引，方便绘图
    results_df.set_index('Model', inplace=True)

    # 提取数量列用于绘图
    count_columns = [f'>={t}' for t in thresholds]
    count_data = results_df[count_columns]
    return count_data

In [ ]:
# get_find_smiles_rank_in_sampling
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
merged_result_pd_list = []
for round_num in [1,2,3]:
    # round_num=1
    all_result_path = f'{all_result_dir}/Round{round_num}'
    all_results = os.listdir(all_result_path)
    all_results_pd = []

    for temp_path in all_results:
        model_name = os.path.splitext(temp_path)[0]
        temp_path = os.path.join(all_result_path, temp_path)
        temp_pd = pd.read_csv(temp_path)
        temp_pd['ModelName'] = [model_name]*len(temp_pd)
        all_results_pd.append(temp_pd)
    merged_result_pd = pd.concat(all_results_pd,axis = 0)#['ModelName'].value_counts()
    merged_result_pd['Finded_Smiles'] = merged_result_pd['Finded_Smiles'].apply(lambda x: [i[1:-1] for i in x[1:-1].split(', ')])
    # merged_result_pd['Finded_Smiles_Num'] = merged_result_pd['Finded_Smiles'].apply(lambda x : len([for i in x if i]))
    merged_result_pd['Finded_Smiles_Affinity'] = merged_result_pd.apply(lambda x: get_find_smiles_affinity(x,affinity_info_map),axis = 1)
    merged_result_pd['Finded_Smiles_Affinity'] = merged_result_pd['Finded_Smiles_Affinity'].apply(lambda x: [i for i in x if i!='None' and i!=np.nan])
    merged_result_pd['Finded_Smiles_Num'] = merged_result_pd['Finded_Smiles_Affinity'].apply(lambda x : len(x))

    merged_result_pd['Protein_in_Crossdock2020'] = merged_result_pd['UniprotID'].apply(lambda x: x.split('_')[0] in crossdock2020_duplicated_uniprotId_map_smiles_in_trainset)
    def replace_model_name(model_name):
        if 'diffSBDD_cond_crossdocked' in model_name:
            return 'DiffSBDD-C'
        elif 'diffSBDD_cond_moad' in model_name:
            return 'DiffSBDD-M'
        elif 'shepherd' in model_name:
            return 'ShEPhERD'
        elif 'DeleteHit2Lead' in model_name:
            return 'Delete'
        else:
            return model_name
    merged_result_pd['ModelName'] = merged_result_pd['ModelName'].apply(lambda x:replace_model_name(x))
    merged_result_pd_list.append(merged_result_pd)

In [ ]:
count_data_list = []
for round_num in [1,2,3]:
    merge_all_affinity_pd = get_protein_type_data(merged_result_pd_list[round_num-1],protein_type='all')
    # merge_all_affinity_pd
    count_data = get_count_data(merge_all_affinity_pd,col_name='Finded_Smiles_Affinity')
    count_data_list.append(count_data)

In [ ]:
def plot_count_bar(count_data,title_name='Count of Affinity Values Above Different Thresholds by Model',protein_type = 'all'):
    # 精确控制版本
    plt.figure(figsize=(10, 6))
    # 设置柱状图的位置和宽度
    thresholds = [0.5, 0.8, 0.9, 0.95, 0.99]
    x = np.arange(len(thresholds))
    width = 0.12  # 使用更小的宽度
    # 计算柱子组的总宽度，确保不会超出边界
    models = count_data.reset_index()['Model'].tolist()
    count_columns = [f'>={t}' for t in thresholds]
    total_width = width * len(models)
    margin = 0.1  # 固定边距
    # 定义背景颜色
    background_colors = ['#f0f8ff', '#fff0f5', '#f0fff0', '#fff8dc', '#f5f5f5']

    # 为每个阈值添加背景色
    # for i, threshold in enumerate(thresholds):
        # plt.axvspan(i - 0.5, i + 0.5, facecolor=background_colors[i % len(background_colors)], alpha=0.8)

    # 为每个模型绘制一组柱状图
    for i, model in enumerate(models):
        model_counts = [count_data.loc[model, col] for col in count_columns]
        # 使用固定边距
        bars = plt.bar(x + margin + i * width, model_counts, width, label=model)
        
        # 在每个柱子上添加数量标签
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.2f}',
                    ha='center', va='bottom', fontsize=14,rotation=90)

    # 在x轴下方添加模型名称标签
    # for i, model in enumerate(models):
    #     for j, threshold in enumerate(thresholds):
    #         plt.text(j + margin + i * width-0.05, -max(count_data.max())*0.01,
    #                 model,
    #                 ha='center', va='top', fontsize=10, rotation=45)

    plt.xlabel('Affinity Threshold',fontsize = 14)
    plt.ylabel('Count',fontsize = 14)
    plt.title(title_name,fontsize = 14)

    # 调整x轴刻度位置，使其与柱子组对齐
    plt.xticks(x + margin + width * (len(models)-1)/2, [f'≥{t}' for t in thresholds],fontsize = 14)

    # 调整y轴范围，为模型名称标签留出空间
    y_max = count_data.max().max()
    # plt.ylim(-y_max*0.15, y_max*1.1)
    plt.yticks([i for i in range(0,int(y_max) +20,10)],fontsize = 14)
    # # 添加图例
    plt.legend(loc='upper right',fontsize = 14)

    plt.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.savefig(f'{save_dir}/{title_name}_{protein_type}.svg', bbox_inches='tight', dpi=660, format='svg')
    plt.show()

In [ ]:
for protein_type in ['all','seen','unseen']:
    for col_name in ['Finded_Smiles_Affinity_max_in_series','Finded_Smiles_Affinity']:
        #Finded_Smiles_Affinity_max_in_series,Finded_Smiles_Affinity
        count_data_list = []
        for round_num in [1,2,3]:
            
            merge_all_affinity_pd = get_protein_type_data(merged_result_pd_list[round_num-1],protein_type=protein_type)
            # merge_all_affinity_pd
            count_data = get_count_data(merge_all_affinity_pd,col_name=col_name)
            count_data_list.append(count_data)
        count_data_mean = pd.concat(count_data_list).groupby('Model').agg(np.mean)
        # count_data
        flag_string = '(Top-1 Affinity Per Series)' if col_name =='Finded_Smiles_Affinity_max_in_series' else ''
        plot_count_bar(count_data_mean,title_name=f'Count of Affinity Values Above Different Thresholds by Model{flag_string}',protein_type = protein_type)
        count_data_mean